In [5]:
import pandas as pd
import os

In [6]:
EXAMPLE = "AAPL"
news_path = os.path.join("./", "news_sentiments.csv")
stocks_path = os.path.join("./", "results2", EXAMPLE + ".csv")
news = pd.read_csv(news_path)
news.tail()

,ticker,title,description,pub_date,link,author,polarity_title,subjectivity_title,polarity_description,subjectivity_description
1793,YELP,'This is America taking a big step backward': ...,A number of major corporations have already ta...,"June 24, 2022 at 12:16 PM",https://finance.yahoo.com/news/this-is-america...,Yahoo Finance,0.000,0.100000,0.182567,0.392550
1794,YELP,Abortion: How corporate America is responding ...,Yahoo Finance's Allie Canal joins the Live sho...,"June 24, 2022 at 2:27 PM",https://finance.yahoo.com/video/abortion-corpo...,Yahoo Finance Video,0.000,0.000000,0.180139,0.416358
1795,YELP,CDC panel backs Moderna vaccine for children a...,CDC panel backs Moderna vaccine for children a...,"Fri, 24 Jun 2022 15:03:00 GMT",https://www.marketwatch.com/story/cdc-panel-ba...,MarketWatch,0.075,0.366667,0.075000,0.366667
1796,YELP,"Disney, Amazon, Netflix among companies offeri...",Yahoo Finance's Allie Garfinkle discusses how ...,"June 24, 2022 at 11:04 AM",https://finance.yahoo.com/video/disney-amazon-...,Yahoo Finance Video,0.000,0.000000,0.132656,0.404388
1797,YELP,How the Supreme Court Roe v. Wade decision imp...,Yahoo Finance's Allie Canal breaks down the ec...,"June 24, 2022 at 10:22 AM",https://finance.yahoo.com/video/supreme-court-...,Yahoo Finance Video,0.000,0.000000,0.099260,0.424278
